In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
name = '4_H12'

df = pd.read_csv('EURUSD_'+name+'_hazirVeri_yeni.csv', parse_dates=['Date'], index_col=0)
print(df.shape)  # (123, 8)
print(type(df['Date']))
df.tail()

(11943, 5)
<class 'pandas.core.series.Series'>


,Date,Open,High,Low,Close
Index,,,,,
11938,2021-12-22 12:00:00,1.12827,1.13423,1.12812,1.13258
11939,2021-12-23 00:00:00,1.13246,1.13422,1.13106,1.13207
11940,2021-12-23 12:00:00,1.13204,1.13378,1.12902,1.13263
11941,2021-12-24 00:00:00,1.13260,1.13415,1.13044,1.13405
11942,2021-12-24 12:00:00,1.13405,1.13435,1.13062,1.13197


In [3]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

adf_test(df['Date'])
adf_test(df['Close'])
adf_test(df['Open'])
adf_test(df['High'])
adf_test(df['Low'])

Augmented Dickey-Fuller Test: 


TypeError: float() argument must be a string or a number, not 'Timestamp'

In [3]:
df_difference = df.diff()
adf_test(df_difference['Close'])

Augmented Dickey-Fuller Test: 
ADF test statistic     -1.856250e+01
p-value                 2.086680e-30
# lags used             3.200000e+01
# observations          1.190900e+04
critical value (1%)    -3.430899e+00
critical value (5%)    -2.861783e+00
critical value (10%)   -2.566899e+00
Strong evidence against the null hypothesis
Reject the null hypothesis
Data has no unit root and is stationary


In [4]:
test_obs = 3583
train = df_difference[:-test_obs]
test = df_difference[-test_obs:]

In [10]:
train['Close'] = np.array(train['Close'], dtype=np.float64)
test['Close'] = np.array(test['Close'], dtype=np.float64)

C:\Users\ertad\AppData\Local\Temp/ipykernel_11908/906253861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Close'] = np.array(train['Close'], dtype=np.float64)
C:\Users\ertad\AppData\Local\Temp/ipykernel_11908/906253861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Close'] = np.array(test['Close'], dtype=np.float64)


In [ ]:
for i in [1,2,3]:
    model = VAR(train)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()